In [21]:
import pandas as pd
import numpy as np
import yfinance as yf
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3 import A2C, SAC , PPO, TD3
from pypfopt.efficient_frontier import EfficientFrontier
from finrl.config import INDICATORS, TRAINED_MODEL_DIR
from config import TRADE_CSV, AGGREGATED_RISK_SCORE
from custom_env import RiskAwareStockTradingEnv
from custom_env2 import RiskAwareStockTradingEnv1


def load_trade():
    trade = pd.read_csv(TRADE_CSV)
    trade = trade.set_index(trade.columns[0])
    trade.index.names = ['']
    
    return trade


def load_trained_a2c():
    trained_a2c = A2C.load(TRAINED_MODEL_DIR + "/agent_a2c")
    
    return trained_a2c


def load_trained_sac():
    trained_sac = SAC.load(TRAINED_MODEL_DIR + "/agent_sac")
    
    return trained_sac

def load_trained_ppo():
    trained_ppo = PPO.load(TRAINED_MODEL_DIR + "/agent_ppo")
    
    return trained_ppo

def load_trained_td3():
    trained_td3 = TD3.load(TRAINED_MODEL_DIR + "/agent_td3")
    
    return trained_td3


def load_aggregated_risk_score(trade):
    sentiment_df = pd.read_csv(AGGREGATED_RISK_SCORE)
        
    # Rename columns for consistency
    sentiment_df = sentiment_df.rename(columns={"datetime": "date", "avg_weighted_score": "risk_score"})

    # Convert to datetime and localize to UTC
    sentiment_df['date'] = pd.to_datetime(sentiment_df['date']).dt.tz_localize('UTC')

    # Merge with UTC-aligned trade['date']
    trade_copy = trade.copy()
    trade_copy['date'] = pd.to_datetime(trade_copy['date'], utc=True)
    sentiment_df['date'] = pd.to_datetime(sentiment_df['date'], utc=True)
    trade_sentiment = pd.merge(trade_copy, sentiment_df, on='date', how='left')

    # Fill missing risk scores with 0
    trade_sentiment['risk_score'] = trade_sentiment['risk_score'].fillna(0)

    return trade_sentiment



In [22]:
trade = load_trade()

a = load_aggregated_risk_score(trade)

print(a.head())


                       date   tic        open        high         low  \
0 2025-01-30 19:30:00+00:00  NVDA  121.454300  123.480003  121.000000   
1 2025-01-30 20:30:00+00:00  NVDA  123.410004  125.000000  121.930000   
2 2025-01-31 14:30:00+00:00  NVDA  123.809998  126.629997  123.209999   
3 2025-01-31 15:30:00+00:00  NVDA  126.180000  127.849998  125.750000   
4 2025-01-31 16:30:00+00:00  NVDA  126.739998  126.970001  124.059998   

        close      volume  day      macd     boll_ub     boll_lb     rsi_30  \
0  123.415001  43926561.0  3.0 -2.774940  128.794814  117.174675  42.739174   
1  124.910004  47647645.0  3.0 -2.330721  128.983218  117.300272  44.619468   
2  126.185997  84699752.0  4.0 -1.854337  129.293092  117.531998  46.179891   
3  126.750000  58838231.0  4.0 -1.414978  129.406299  117.491792  46.864474   
4  125.309998  44879038.0  4.0 -1.169498  129.304382  117.515709  45.341194   

       cci_30      dx_30  close_30_sma  close_60_sma  risk_score  
0  -20.626883  12.4

In [ ]:
def load_sentiment_and_turbulence(trade, turbulence_path='turbulence.csv'):
    # Get trade with risk_score
    trade_sentiment = load_aggregated_risk_score(trade)
    
    # Load turbulence data
    turbulence_df = pd.read_csv(turbulence_path)
    if 'date' not in turbulence_df.columns:
        raise ValueError("'date' column not found in turbulence data.")
    if 'turbulence_bin' not in turbulence_df.columns:
        raise ValueError("'turbulence_bin' column not found in turbulence data.")
    turbulence_df['date'] = pd.to_datetime(turbulence_df['date'], utc=True)
    turbulence_df['turbulence_bin'] = turbulence_df['turbulence_bin'].fillna('low')
    
    # Merge on 'date'
    merged = pd.merge(trade_sentiment, turbulence_df[['date', 'turbulence_bin']], on='date', how='left')
    merged['turbulence_bin'] = merged['turbulence_bin'].fillna('low')
    return merged

# Example usage:
trade = load_trade()
trade_with_risk_and_turb = load_sentiment_and_turbulence(trade, turbulence_path='turbulence.csv')
print(trade_with_risk_and_turb)


                         date   tic        open        high         low  \
0   2025-01-30 19:30:00+00:00  NVDA  121.454300  123.480003  121.000000   
1   2025-01-30 20:30:00+00:00  NVDA  123.410004  125.000000  121.930000   
2   2025-01-31 14:30:00+00:00  NVDA  123.809998  126.629997  123.209999   
3   2025-01-31 15:30:00+00:00  NVDA  126.180000  127.849998  125.750000   
4   2025-01-31 16:30:00+00:00  NVDA  126.739998  126.970001  124.059998   
..                        ...   ...         ...         ...         ...   
563 2025-05-28 14:30:00+00:00  NVDA  135.259995  135.630005  134.800003   
564 2025-05-28 15:30:00+00:00  NVDA  135.595001  136.100006  135.479996   
565 2025-05-28 16:30:00+00:00  NVDA  135.934998  136.500000  135.520004   
566 2025-05-28 17:30:00+00:00  NVDA  136.449997  137.250000  136.240005   
567 2025-05-28 18:30:00+00:00  NVDA  136.660004  137.190002  136.315002   

          close      volume  day      macd     boll_ub     boll_lb     rsi_30  \
0    123.415001  4